Q1. Running Elastic

In [1]:
import requests

url = 'http://localhost:9200'
response = requests.get(url)
response.json()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-02 10:42:03--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.6’

minsearch.py.6      100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-07-02 10:42:04 (7.26 MB/s) - ‘minsearch.py.6’ saved [3832/3832]



In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

Q2. Indexing the data

In [5]:
from elasticsearch import Elasticsearch
import minsearch
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
q="How do I execute a command in a running docker container?"

In [7]:
index.fit(documents)

In [8]:
def search(query):
    boost = {'question': 4.0}

    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=5
    )

    return results

In [9]:
from openai import OpenAI
client = OpenAI()

In [10]:
es_client = Elasticsearch('http://localhost:9200') 

In [11]:
es_client

<Elasticsearch(['http://localhost:9200'])>

In [12]:
from tqdm.auto import tqdm

In [13]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "homework-intro"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'homework-intro'})

In [14]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [117]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
query = "How do I execute a command in a running docker container?"

In [120]:
def elastic_search(query):
    search_query = {
        "size": 1,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_score'])
    
    return result_docs

In [121]:
elastic_search(query)

[84.050095]

In [16]:
def elastic_search_Q4(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [17]:
result = elastic_search_Q4(query)

In [20]:
question = result[2]['question']

In [21]:
question

'How do I copy files from a different folder into docker container’s working directory?'

In [23]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
Q:{question}

A:
{context}
""".strip()
    context = ""
    for doc in search_results:
        context = context + f"Q:{doc['question']}\nA:{doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [24]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    # answer = llm(prompt)
    return prompt

In [27]:
Q5 = rag(question)

In [29]:
len(Q5)

2471

In [31]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [32]:
len(encoding.encode(Q5))

540